<a href="https://colab.research.google.com/github/mirovix/CV_final_project/blob/main/NN_boats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import io
from google.colab import drive
from google.colab import files
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
drive.mount('/content/drive')

with open('/content/drive/MyDrive/dataset_boats/label.json') as json_file:
    json_file = json.load(json_file)

#print(len(json_file[30]['Label']))

x_start, y_start, x_end, y_end, name_pic = [],[],[],[],[]
for row in json_file:
  if(len(row['Label']) == 0):
    continue
  name_pic.append(row['External ID'])
  x_start.append(float(row['Label']['objects'][0]['bbox']['left']))
  y_start.append(float(row['Label']['objects'][0]['bbox']['top']))
  x_end.append(float(row['Label']['objects'][0]['bbox']['left'] + row['Label']['objects'][0]['bbox']['width']))
  y_end.append(float(row['Label']['objects'][0]['bbox']['top'] + row['Label']['objects'][0]['bbox']['height']))

i=0
width = 32
height = 32
data,targets = [],[]
for image in glob.glob("/content/drive/MyDrive/dataset_boats/train/*.jpg"):
   cv_image = cv2.imread(image)
   cv_image = cv2.resize(cv_image, (height, width))
   data.append(img_to_array(cv_image))
   targets.append((x_start[i], y_start[i], x_end[i], y_end[i]))
   #h.append(cv_image.shape[0])
   #w.append(cv_image.shape[1])
   if(i==300):
    break
   i=i+1

print(len(data))

In [ ]:
	# scale the bounding box coordinates relative to the spatial
	# dimensions of the input image
for i in range(len(data)):
  x_start[i] = x_start[i] / w[i]
  y_start[i] = y_start[i] / h[i]
  x_end[i] = x_end[i] / w[i]
  y_end[i] = y_end[i] / h[i]

In [ ]:
targets = []
i=0
for image in glob.glob("/content/drive/MyDrive/dataset_boats/train/*.jpg"):
  image_loaded = load_img(image, target_size=(224, 224))
  data.append(img_to_array(image_loaded))
  targets.append((x_start[i], y_start[i], x_end[i], y_end[i]))
  if(i==13):
    break
  i=i+1

In [ ]:
# convert the data and targets to NumPy arrays, scaling the input
# pixel intensities from the range [0, 255] to [0, 1]
#data = np.array(data, dtype="float32") / 255.0
#targets = np.array(targets, dtype="float32")
# partition the data into training and testing splits using 90% of
# the data for training and the remaining 10% for testing
data = np.array(data, dtype="float32") / 255.0
targets = np.array(targets, dtype="float32")
X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=0.10, random_state=42)

In [ ]:
def plot_loss(history):
  plt.figure(figsize=(10,6))
  plt.plot(history.epoch,history.history['loss'])
  plt.plot(history.epoch,history.history['val_loss'])
  plt.title('loss')

def plot_accuracy(history):
  plt.figure(figsize=(10,6))
  plt.plot(history.epoch,history.history['accuracy'])
  plt.plot(history.epoch,history.history['val_accuracy'])
  plt.title('accuracy')

In [ ]:
print(y_train[0].shape)
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=[3,3], activation='tanh', input_shape=[32, 32, 3]),
    keras.layers.MaxPool2D(pool_size=[2,2]),
    keras.layers.Conv2D(filters=32, kernel_size=[2,2], activation='tanh'),
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation="sigmoid")
  ])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
print(model.summary())

history = model.fit(X_train, y_train, epochs=20, batch_size=256 ,validation_data=(X_test, y_test))

plot_loss(history)
plot_accuracy(history)

scores = model.evaluate(X_test, y_test, verbose=2)
print(" %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("----------------------------")